## This script generates a video from a sequence of pictures.


In [1]:
import cv2
import os, sys
from tqdm import tqdm
from videolib import initVideoWriter, imageHiveOverview
import multiprocessing
import pandas as pd
pd.set_option('display.width', 1000)
sys.path.append("..")
from libimage import fetchImagesPaths

## Configuration

In [2]:
# Path to the folder containing the pictures
rootpath = '/Users/cyrilmonette/Library/CloudStorage/SynologyDrive-data/25.07_aSensing_OH/Images/'
rootpath_img_period = 5 # Period between every image in the folder in minutes
hive = "1"

# The following are in CET time zone:
first_dt_str = "250722-234000"
last_dt_str = "250723-161000"

frame_drop = 1 # We keep 1 frame every frame_drop frames. Put one to keep all frames.
fps_video = 10 # Frames per second for the video

## Main code

In [ ]:
first_dt = pd.to_datetime(first_dt_str, format="%y%m%d-%H%M%S").tz_localize('CET')
last_dt = pd.to_datetime(last_dt_str, format="%y%m%d-%H%M%S").tz_localize('CET')
assert first_dt < last_dt, "First date must be before last date"
print("First date: ", first_dt)
print("Last date: ", last_dt)

time_range = pd.date_range(start=first_dt, end=last_dt, freq=f"{rootpath_img_period*frame_drop}min")
time_range = time_range.to_list()
print("Length time range: ", len(time_range))
img_paths = fetchImagesPaths(rootpath, time_range, hive, invalid_recovery_time=5,images_fill_limit=None)
print(img_paths)

# Find the first index of img_paths that does not contain Nones in any column
first_valid_index = img_paths.dropna(how='all', inplace=False).index[0]
example_frame = imageHiveOverview([cv2.imread(img_paths.loc[first_valid_index].iloc[j]) for j in range(4)],
                                    ["hive" + hive + "_rpi" + str(j+1) + "_" + img_paths.loc[first_valid_index].iloc[j] for j in range(4)])
name = "hive" + hive + "_" + first_dt.tz_convert('UTC').strftime("%y%m%d-%H%M%SZ") + "_" + last_dt.tz_convert('UTC').strftime("%y%m%d-%H%M%SZ")
video = initVideoWriter(dest="outputVideos/", frame=example_frame,name=name, fps=fps_video, grayscale=False)
print("Writing video...")

# Iterate over img_paths DataFrame
for i, row in tqdm(img_paths.iterrows(), total=img_paths.shape[0]):
    imgs = []
    names = []
    validity = True 
    if 'valid' in row:
        validity = row['valid']
        row.drop('valid', inplace=True)

    for j in range(len(row)):
        if row.iloc[j] is not None:
            file_path = row.iloc[j]
            names.append(os.path.basename(file_path))  # Get the file name
            imgs.append(cv2.imread(file_path))
        else:
            # If the file does not exist, append a black image
            names.append("No image available")
            # Append a black image of the same size as the example frame
            imgs.append(cv2.imread(img_paths.loc[first_valid_index].iloc[j]) * 0)

    assembled_img = imageHiveOverview(imgs, names, dt=i, valid=validity)
    video.write(assembled_img)

# Release the VideoWriter object
video.release()
print("Video written successfully.")

First date:  2025-07-22 23:40:00+02:00
Last date:  2025-07-23 16:10:00+02:00
Length time range:  199
                                                                        h1r1                                               h1r2                                               h1r3                                               h1r4  valid
2025-07-22 23:40:00+02:00  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  False
2025-07-22 23:45:00+02:00  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  False
2025-07-22 23:50:00+02:00  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/c

100%|██████████| 199/199 [00:42<00:00,  4.71it/s]

Video written successfully.
